In [ ]:
%%capture
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1vTI1hkI5QV26zLYOo7q5piI3UEhvFSbg' -O nbody.zip
!unzip nbody.zip
!mv nbody-python-master/* .

In [ ]:
!ls

LICENSE   nbody.ipynb  nbody.py		    nbody.zip	 README.md
__MACOSX  nbody.png    nbody-python-master  __pycache__  sample_data


In [ ]:
!nvidia-smi

Mon Aug 22 14:20:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Tensorflow implementation

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
# Simulation parameters
N = 1000  # Number of particles
t = 0  # current time of the simulation
tEnd = 10.0  # time at which simulation ends
dt = 0.01  # timestep
softening = 0.1  # softening length
G = 1.0  # Newton's Gravitational Constant
plotRealTime = True  # switch on for plotting as the simulation goes along

In [ ]:
# Generate Initial Conditions
np.random.seed(17)  # set the random number generator seed

mass = 20.0 * np.ones((N, 1)) / N  # total mass of particles is 20
mass_sq = np.squeeze(mass)
pos = np.random.randn(N, 3)  # randomly selected positions and velocities
vel = np.random.randn(N, 3)

# Convert to Center-of-Mass frame
vel -= np.mean(mass * vel, 0) / np.mean(mass)

In [ ]:
pos_tf = tf.convert_to_tensor(pos)
mass_tf = tf.convert_to_tensor(mass)
G_tf = tf.convert_to_tensor(G, dtype=tf.float64)
softening_tf = tf.convert_to_tensor(softening, dtype=tf.float64)

In [ ]:
def get_acc_tf(pos_tf, mass_tf):
    # positions r = [x,y,z] for all particles
    x = pos_tf[:, 0:1]
    y = pos_tf[:, 1:2]
    z = pos_tf[:, 2:3]

    # matrix that stores all pairwise particle separations: r_j - r_i
    dx = tf.transpose(x) - x
    dy = tf.transpose(y) - y
    dz = tf.transpose(z) - z

    inv_r3 = (dx ** 2 + dy ** 2 + dz ** 2 + softening_tf ** 2) ** (-1.5)

    ax = G_tf * (dx * inv_r3) @ mass_tf
    ay = G_tf * (dy * inv_r3) @ mass_tf
    az = G_tf * (dz * inv_r3) @ mass_tf

    return tf.concat([ax, ay, az], axis=1)

get_acc_tf_graph = tf.function(get_acc_tf)

In [ ]:
%%timeit
b = get_acc_tf_graph(pos_tf, mass_tf)

615 µs ± 154 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
